In [15]:
d = '/media/shreyas/DATA/ML_DATA/wheelai/gtav/train/'
#d = '/media/shreyas/DATA/ML_DATA/wheelai/gtav/sample/'

x_trn_path = '/home/shreyas/Downloads/data/x_trn.bc'
y_trn_path = d + 'y_trn.bc'

s_crp_path = d + 's_crp.bc'
x_crp_path = '/home/shreyas/Downloads/data/x_crp.bc'
v_crp_path = '/home/shreyas/Downloads/data/v_crp.bc'

x_val_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtav/valid/x_val.bc'
y_val_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtav/valid/y_val.bc'
results_path = '/media/shreyas/DATA/ML_DATA/wheelai/gtav/results/'

In [16]:
from __future__ import division, print_function

import os, json
from glob import glob
import numpy as np
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom

import keras
from keras.callbacks import ModelCheckpoint
from keras import backend as K
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, load_model,  Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.preprocessing import image
from keras.applications import VGG16
from keras.layers import Input, Lambda, Cropping2D, Activation, ELU
from keras.layers.merge import add, concatenate

%matplotlib inline

In [17]:
import bcolz

In [18]:
x_trn = bcolz.open(x_trn_path)
y_trn = bcolz.open(y_trn_path)
x_val = bcolz.open(x_val_path)
y_val = bcolz.open(y_val_path)

In [19]:
print(x_trn.shape, y_trn.shape, x_val.shape, y_val.shape)

(545676, 160, 320, 3) (545676, 3) (18885, 160, 320, 3) (18885, 3)


In [20]:
batch_size = 128
input_shape = (160, 320, 3)

In [21]:
from sklearn.utils import shuffle

def get_train_batch(offset, batch_size):
    end = offset + batch_size
    return x_trn[offset:end], y_trn[offset:end]
            
def get_valid_batch(offset, batch_size):
    end = offset + batch_size
    return x_val[offset:end], y_val[offset:end]
        
def generate_batch(num_samples, batch_size=32, train=True):
    """generator return train data and valid data in batches"""
    # Loop forever so the generator never terminates 
    while 1:
        #x_trn, y_trn = shuffle(y_trn, y_trn)
        for offset in range(0, num_samples, batch_size):
            if train:
                yield (get_train_batch(offset, batch_size))
            else:
                yield (get_valid_batch(offset, batch_size))
            

### More of Nvidia's [End to End Learning Model](https://devblogs.nvidia.com/parallelforall/deep-learning-self-driving-cars/)

In [22]:
def nvidia_model():
    '''Implements Nvidia's end to end learning model'''
    # Croping the image to retain only road part of image
    x = Input(shape=input_shape)
    # layer to learn color space
    color_s = Conv2D(3, (1,1), padding='same')(x)
    # normalise to mean 0
    norm = Lambda(lambda x: (x / 255.0) - 0.5)(color_s)
    
    conv_b1 = Conv2D(24, (3,3), padding='same', activation='relu')(norm)
    batch_1 = BatchNormalization()(conv_b1)
    max_pl1 = MaxPooling2D()(batch_1)
    
    conv_b2 = Conv2D(36, (3,3), padding='same', activation='relu')(max_pl1)
    batch_2 = BatchNormalization()(conv_b2)
    max_pl2 = MaxPooling2D()(batch_2)
    
    conv_b3 = Conv2D(48, (3,3), padding='same', activation='relu')(max_pl2)
    batch_3 = BatchNormalization()(conv_b3)
    max_pl3 = MaxPooling2D()(batch_3)
    
    conv_b4 = Conv2D(64, (3,3), padding='same', activation='relu')(max_pl3)
    batch_4 = BatchNormalization()(conv_b4)
    max_pl4 = MaxPooling2D()(batch_4)
    dropout = Dropout(0.1)(max_pl4)
    
    conv_b5 = Conv2D(64, (3,3), padding='same', activation='relu')(dropout)
    batch_5 = BatchNormalization()(conv_b5)
    max_pl5 = MaxPooling2D((1,4))(batch_5)
    dropout = Dropout(0.1)(max_pl5)
    
    flatten = Flatten()(dropout)
    fc1 = Dense(100, activation='relu')(flatten)
    batch_fc1 = BatchNormalization()(fc1)
    drop_fc1 = Dropout(0.2)(batch_fc1)
    
    fc2 = Dense(50, activation='relu')(drop_fc1)
    batch_fc2 = BatchNormalization()(fc2)
    drop_fc2 = Dropout(0.3)(batch_fc2)
    
    fc3 = Dense(10, activation='relu')(drop_fc2)
    batch_fc3 = BatchNormalization()(fc3)
    drop_fc3 = Dropout(0.5)(batch_fc3)
    
    output = Dense(3)(drop_fc3)
    
    model = Model(inputs=x, outputs=output)
    return model

In [23]:
def nvidia_conv():
    '''Implement a model similar to Nvidia end to end but with only conv layers'''
    model = Sequential()
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=input_shape))
    model.add(Conv2D(3, (1,1), padding='same')) 
    model.add(Conv2D(24, (3,3), padding='same', activation='elu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Conv2D(36, (3,3), padding='same', activation='elu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D())   
    model.add(Conv2D(48, (3,3), padding='same', activation='elu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Conv2D(64, (3,3), padding='same', activation='elu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D())
    model.add(Conv2D(128, (3,3), padding='same', activation='elu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D((1,4)))
    model.add(Conv2D(3, (3,3), padding='same'))
    model.add(GlobalAveragePooling2D())
    model.add(Dense(3))
    
    return model

In [24]:
def commaai():
    #ch, row, col = 3, 160, 320  # camera format

    model = Sequential()
    model.add(Lambda(lambda x: x/127.5 - 1.,input_shape=input_shape))
    model.add(Conv2D(16, (8, 8), strides=(4, 4), padding="same"))
    #model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(MaxPooling2D())
    model.add(Conv2D(32, (5, 5), strides=(2, 2), padding="same"))
    model.add(Dropout(.1))
    #model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(MaxPooling2D((1,2)))
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding="same"))
    model.add(Dropout(.2))
    #model.add(BatchNormalization())
    model.add(Activation('elu'))
    model.add(MaxPooling2D())
    model.add(Flatten())
    model.add(Dropout(.3))
    model.add(Dense(512))
    model.add(Dropout(.5))
    model.add(ELU())
    model.add(Dense(3))

    model.compile(optimizer="adam", loss="mse")

    return model


## 1 CommaAI Model

In [25]:
model = commaai()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_2 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 40, 80, 16)        3088      
_________________________________________________________________
activation_4 (Activation)    (None, 40, 80, 16)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 20, 40, 16)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 10, 20, 32)        12832     
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 20, 32)        0         
_________________________________________________________________
activation_5 (Activation)    (None, 10, 20, 32)        0         
__________

In [26]:
# serialize model to JSON
model_path = results_path+'commaai.json'
model_json = model.to_json()
with open(model_path, "w") as json_file:
    json_file.write(model_json)

print("Saved model to disk")

Saved model to disk


In [27]:
trn_gen = generate_batch(x_trn.shape[0], batch_size)
val_gen = generate_batch(x_val.shape[0], batch_size, train=False)

In [28]:
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai1.h5')

Epoch 1/10
4264/4263 [==============================] - 400s - loss: 0.0611 - val_loss: 0.3931
Epoch 2/10
4264/4263 [==============================] - 459s - loss: 0.0771 - val_loss: 1.0703
Epoch 3/10
4264/4263 [==============================] - 387s - loss: 0.0673 - val_loss: 1.0616
Epoch 4/10
4264/4263 [==============================] - 356s - loss: 0.0793 - val_loss: 0.9807
Epoch 5/10
4264/4263 [==============================] - 352s - loss: 0.0684 - val_loss: 1.0060
Epoch 6/10
4264/4263 [==============================] - 422s - loss: 0.0694 - val_loss: 1.0194
Epoch 7/10
4264/4263 [==============================] - 430s - loss: 0.0683 - val_loss: 1.0063
Epoch 8/10
4264/4263 [==============================] - 359s - loss: 0.0678 - val_loss: 1.0044
Epoch 9/10
4264/4263 [==============================] - 306s - loss: 0.0674 - val_loss: 0.9490
Epoch 10/10
4264/4263 [==============================] - 292s - loss: 0.0678 - val_loss: 0.9587


In [29]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai2.h5')

Epoch 1/10
4264/4263 [==============================] - 305s - loss: 0.0674 - val_loss: 0.8246
Epoch 2/10
4264/4263 [==============================] - 326s - loss: 0.0672 - val_loss: 0.4265
Epoch 3/10
4264/4263 [==============================] - 327s - loss: 0.0672 - val_loss: 0.6218
Epoch 4/10
4264/4263 [==============================] - 373s - loss: 0.0673 - val_loss: 0.1989
Epoch 5/10
4264/4263 [==============================] - 410s - loss: 0.0673 - val_loss: 0.2903
Epoch 6/10
4264/4263 [==============================] - 434s - loss: 0.0674 - val_loss: 0.3024
Epoch 7/10
4264/4263 [==============================] - 438s - loss: 0.0674 - val_loss: 0.1716
Epoch 8/10
4264/4263 [==============================] - 438s - loss: 0.0674 - val_loss: 0.1121
Epoch 9/10
4264/4263 [==============================] - 435s - loss: 0.0676 - val_loss: 0.0608
Epoch 10/10
4264/4263 [==============================] - 417s - loss: 0.0677 - val_loss: 0.0506


In [30]:
model.optimizer.lr = 1e-3
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai3.h5')

Epoch 1/10
4264/4263 [==============================] - 303s - loss: 0.0677 - val_loss: 0.5960
Epoch 2/10
4264/4263 [==============================] - 320s - loss: 0.0677 - val_loss: 0.6959
Epoch 3/10
4264/4263 [==============================] - 316s - loss: 0.0677 - val_loss: 0.4711
Epoch 4/10
4264/4263 [==============================] - 299s - loss: 0.0677 - val_loss: 0.4745
Epoch 5/10
4264/4263 [==============================] - 296s - loss: 0.0679 - val_loss: 0.5044
Epoch 6/10
4264/4263 [==============================] - 293s - loss: 0.0679 - val_loss: 0.5069
Epoch 7/10
4264/4263 [==============================] - 301s - loss: 0.0679 - val_loss: 0.4707
Epoch 8/10
4264/4263 [==============================] - 296s - loss: 0.0679 - val_loss: 0.5075
Epoch 9/10
4264/4263 [==============================] - 342s - loss: 0.0679 - val_loss: 0.3629
Epoch 10/10
4264/4263 [==============================] - 348s - loss: 0.0680 - val_loss: 0.3827


In [31]:
#filepath2 = results_path+'nvidia_aug.h5'
#checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1, save_best_only=False,\
#                             save_weights_only=True, mode='min', period=1)
#callbacks2=[checkpoint2]

In [32]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai4.h5')

Epoch 1/10
4264/4263 [==============================] - 365s - loss: 0.0681 - val_loss: 0.3069
Epoch 2/10
4264/4263 [==============================] - 388s - loss: 0.0681 - val_loss: 0.2275
Epoch 3/10
4264/4263 [==============================] - 394s - loss: 0.0681 - val_loss: 0.3215
Epoch 4/10
4264/4263 [==============================] - 409s - loss: 0.0681 - val_loss: 0.3500
Epoch 5/10
4264/4263 [==============================] - 430s - loss: 0.0681 - val_loss: 0.2233
Epoch 6/10
4264/4263 [==============================] - 372s - loss: 0.0682 - val_loss: 0.2588
Epoch 7/10
4264/4263 [==============================] - 344s - loss: 0.0682 - val_loss: 0.2528
Epoch 8/10
4264/4263 [==============================] - 369s - loss: 0.0683 - val_loss: 0.2318
Epoch 9/10
4264/4263 [==============================] - 383s - loss: 0.0683 - val_loss: 0.4475
Epoch 10/10
4264/4263 [==============================] - 337s - loss: 0.0684 - val_loss: 0.2895


In [33]:
model.optimizer.lr = 1e-4
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai5.h5')

Epoch 1/10
4264/4263 [==============================] - 349s - loss: 0.0683 - val_loss: 0.2431
Epoch 2/10
4264/4263 [==============================] - 351s - loss: 0.0684 - val_loss: 0.2737
Epoch 3/10
4264/4263 [==============================] - 379s - loss: 0.0684 - val_loss: 0.2421
Epoch 4/10
4264/4263 [==============================] - 414s - loss: 0.0685 - val_loss: 0.2384
Epoch 5/10
4264/4263 [==============================] - 413s - loss: 0.0684 - val_loss: 0.2651
Epoch 6/10
4264/4263 [==============================] - 418s - loss: 0.0685 - val_loss: 0.2508
Epoch 7/10
4264/4263 [==============================] - 432s - loss: 0.0686 - val_loss: 0.2447
Epoch 8/10
4264/4263 [==============================] - 433s - loss: 0.0685 - val_loss: 0.2757
Epoch 9/10
4264/4263 [==============================] - 437s - loss: 0.0686 - val_loss: 0.2306
Epoch 10/10
4264/4263 [==============================] - 439s - loss: 0.0685 - val_loss: 0.2830


In [34]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai6.h5')

Epoch 1/10
4264/4263 [==============================] - 336s - loss: 0.0686 - val_loss: 0.2827
Epoch 2/10
4264/4263 [==============================] - 325s - loss: 0.0685 - val_loss: 0.2383
Epoch 3/10
4264/4263 [==============================] - 334s - loss: 0.0687 - val_loss: 0.2990
Epoch 4/10
4264/4263 [==============================] - 347s - loss: 0.0685 - val_loss: 0.2543
Epoch 5/10
4264/4263 [==============================] - 354s - loss: 0.0686 - val_loss: 0.2636
Epoch 6/10
4264/4263 [==============================] - 366s - loss: 0.0686 - val_loss: 0.2645
Epoch 7/10
4264/4263 [==============================] - 365s - loss: 0.0687 - val_loss: 0.2732
Epoch 8/10
4264/4263 [==============================] - 363s - loss: 0.0688 - val_loss: 0.2673
Epoch 9/10
4264/4263 [==============================] - 370s - loss: 0.0687 - val_loss: 0.2859
Epoch 10/10
4264/4263 [==============================] - 371s - loss: 0.0687 - val_loss: 0.2646


In [35]:
model.optimizer.lr = 1e-4
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai7.h5')

Epoch 1/10
4264/4263 [==============================] - 393s - loss: 0.0692 - val_loss: 1.0369
Epoch 2/10
4264/4263 [==============================] - 423s - loss: 0.0687 - val_loss: 1.0897
Epoch 3/10
4264/4263 [==============================] - 434s - loss: 0.0688 - val_loss: 1.1074
Epoch 4/10
4264/4263 [==============================] - 438s - loss: 0.0687 - val_loss: 1.0036
Epoch 5/10
4264/4263 [==============================] - 441s - loss: 0.0687 - val_loss: 0.9539
Epoch 6/10
4264/4263 [==============================] - 421s - loss: 0.0688 - val_loss: 1.0153
Epoch 7/10
4264/4263 [==============================] - 322s - loss: 0.0689 - val_loss: 0.9910
Epoch 8/10
4264/4263 [==============================] - 326s - loss: 0.0688 - val_loss: 1.1065
Epoch 9/10
4264/4263 [==============================] - 315s - loss: 0.0689 - val_loss: 1.1128
Epoch 10/10
4264/4263 [==============================] - 315s - loss: 0.0688 - val_loss: 0.9888


In [36]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'commaai8.h5')

Epoch 1/10
4264/4263 [==============================] - 322s - loss: 0.0690 - val_loss: 0.9896
Epoch 2/10
4264/4263 [==============================] - 340s - loss: 0.0689 - val_loss: 0.9875
Epoch 3/10
4264/4263 [==============================] - 357s - loss: 0.0689 - val_loss: 1.0992
Epoch 4/10
4264/4263 [==============================] - 363s - loss: 0.0690 - val_loss: 1.2068
Epoch 5/10
4264/4263 [==============================] - 344s - loss: 0.0689 - val_loss: 1.0525
Epoch 6/10
4264/4263 [==============================] - 360s - loss: 0.0690 - val_loss: 1.0442
Epoch 7/10
4264/4263 [==============================] - 373s - loss: 0.0690 - val_loss: 1.1774
Epoch 8/10
4264/4263 [==============================] - 384s - loss: 0.0690 - val_loss: 1.1604
Epoch 9/10
4264/4263 [==============================] - 431s - loss: 0.0689 - val_loss: 1.1016
Epoch 10/10
4264/4263 [==============================] - 454s - loss: 0.0690 - val_loss: 1.0779


## 2 Nvidia Model

In [37]:
model = nvidia_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 160, 320, 3)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 160, 320, 3)       12        
_________________________________________________________________
lambda_3 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 160, 320, 24)      672       
_________________________________________________________________
batch_normalization_1 (Batch (None, 160, 320, 24)      96        
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 80, 160, 24)       0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 80, 160, 36)       7812      
__________

In [38]:
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer=adam,loss='mse')

In [39]:
# serialize model to JSON
model_path = results_path+'nvidia.json'
model_json = model.to_json()
with open(model_path, "w") as json_file:
    json_file.write(model_json)

print("Saved model to disk")

Saved model to disk


In [40]:
trn_gen = generate_batch(x_trn.shape[0], batch_size)
val_gen = generate_batch(x_val.shape[0], batch_size, train=False)

In [41]:
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia1.h5')

Epoch 1/10
4264/4263 [==============================] - 4327s - loss: 1.0730 - val_loss: 2.1095
Epoch 2/10
4264/4263 [==============================] - 4312s - loss: 0.1688 - val_loss: 0.3281
Epoch 3/10
4264/4263 [==============================] - 4319s - loss: 0.0916 - val_loss: 0.1445
Epoch 4/10
4264/4263 [==============================] - 4318s - loss: 0.0860 - val_loss: 0.1770
Epoch 5/10
4264/4263 [==============================] - 4318s - loss: 0.0843 - val_loss: 0.1566
Epoch 6/10
4264/4263 [==============================] - 4318s - loss: 0.0832 - val_loss: 0.1662
Epoch 7/10
4264/4263 [==============================] - 4321s - loss: 0.0822 - val_loss: 0.1997
Epoch 8/10
4264/4263 [==============================] - 4321s - loss: 0.0816 - val_loss: 0.2276
Epoch 9/10
4264/4263 [==============================] - 4320s - loss: 0.0808 - val_loss: 0.2216
Epoch 10/10
4264/4263 [==============================] - 4325s - loss: 0.0802 - val_loss: 0.1388


In [42]:
#filepath1 = results_path+'nvidia.h5'
#checkpoint1 = ModelCheckpoint(filepath1, monitor='val_loss', verbose=1, save_best_only=False,\
#                             save_weights_only=True, mode='min', period=1)
#callbacks1=[checkpoint1]

In [43]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia2.h5')

Epoch 1/10
4264/4263 [==============================] - 4321s - loss: 0.0798 - val_loss: 0.2391
Epoch 2/10
4264/4263 [==============================] - 4322s - loss: 0.0790 - val_loss: 0.2149
Epoch 3/10
4264/4263 [==============================] - 4323s - loss: 0.0785 - val_loss: 0.2168
Epoch 4/10
4264/4263 [==============================] - 4324s - loss: 0.0779 - val_loss: 0.2222
Epoch 5/10
4264/4263 [==============================] - 4322s - loss: 0.0775 - val_loss: 0.2302
Epoch 6/10
4264/4263 [==============================] - 4320s - loss: 0.0771 - val_loss: 0.2397
Epoch 7/10
4264/4263 [==============================] - 4316s - loss: 0.0767 - val_loss: 0.2462
Epoch 8/10
4264/4263 [==============================] - 4312s - loss: 0.0763 - val_loss: 0.2498
Epoch 9/10
4264/4263 [==============================] - 4312s - loss: 0.0760 - val_loss: 0.2339
Epoch 10/10
4264/4263 [==============================] - 4326s - loss: 0.0757 - val_loss: 0.2360


In [44]:
model.optimizer.lr = 1e-3
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia3.h5')

Epoch 1/10
4264/4263 [==============================] - 4326s - loss: 0.0757 - val_loss: 0.2519
Epoch 2/10
4264/4263 [==============================] - 5624s - loss: 0.0752 - val_loss: 0.2491
Epoch 3/10
4264/4263 [==============================] - 7588s - loss: 0.0749 - val_loss: 0.2420
Epoch 4/10
4264/4263 [==============================] - 8345s - loss: 0.0747 - val_loss: 0.2488
Epoch 5/10
4264/4263 [==============================] - 8202s - loss: 0.0745 - val_loss: 0.2523
Epoch 6/10
4264/4263 [==============================] - 6157s - loss: 0.0742 - val_loss: 0.2604
Epoch 7/10
4264/4263 [==============================] - 5637s - loss: 0.0742 - val_loss: 0.2607
Epoch 8/10
4264/4263 [==============================] - 5739s - loss: 0.0739 - val_loss: 0.2613
Epoch 9/10
4264/4263 [==============================] - 5654s - loss: 0.0738 - val_loss: 0.2611
Epoch 10/10
4264/4263 [==============================] - 5703s - loss: 0.0736 - val_loss: 0.2619


In [45]:
#filepath2 = results_path+'nvidia_aug.h5'
#checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1, save_best_only=False,\
#                             save_weights_only=True, mode='min', period=1)
#callbacks2=[checkpoint2]

In [46]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia4.h5')

Epoch 1/10
4264/4263 [==============================] - 5685s - loss: 0.0738 - val_loss: 0.2426
Epoch 2/10
4264/4263 [==============================] - 5671s - loss: 0.0733 - val_loss: 0.2488
Epoch 3/10
4264/4263 [==============================] - 5655s - loss: 0.0732 - val_loss: 0.2439
Epoch 4/10
4264/4263 [==============================] - 5703s - loss: 0.0730 - val_loss: 0.2423
Epoch 5/10
4264/4263 [==============================] - 5670s - loss: 0.0729 - val_loss: 0.2509
Epoch 6/10
4264/4263 [==============================] - 5671s - loss: 0.0727 - val_loss: 0.2506
Epoch 7/10
4264/4263 [==============================] - 5649s - loss: 0.0726 - val_loss: 0.2481
Epoch 8/10
4264/4263 [==============================] - 5663s - loss: 0.0725 - val_loss: 0.2528
Epoch 9/10
4264/4263 [==============================] - 5647s - loss: 0.0723 - val_loss: 0.2516
Epoch 10/10
4264/4263 [==============================] - 5679s - loss: 0.0721 - val_loss: 0.2576


In [47]:
model.optimizer.lr = 1e-4
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia5.h5')

Epoch 1/10
4264/4263 [==============================] - 6357s - loss: 0.0723 - val_loss: 0.2281
Epoch 2/10
4264/4263 [==============================] - 34155s - loss: 0.0719 - val_loss: 0.2297
Epoch 3/10
4264/4263 [==============================] - 29895s - loss: 0.0719 - val_loss: 0.2322
Epoch 4/10
4264/4263 [==============================] - 8122s - loss: 0.0717 - val_loss: 0.2301
Epoch 5/10
4264/4263 [==============================] - 8109s - loss: 0.0716 - val_loss: 0.2304
Epoch 6/10
4264/4263 [==============================] - 8187s - loss: 0.0716 - val_loss: 0.2263
Epoch 7/10
4264/4263 [==============================] - 8165s - loss: 0.0715 - val_loss: 0.2392
Epoch 8/10
4264/4263 [==============================] - 8192s - loss: 0.0714 - val_loss: 0.2363
Epoch 9/10
4264/4263 [==============================] - 8175s - loss: 0.0713 - val_loss: 0.2334
Epoch 10/10
4264/4263 [==============================] - 8162s - loss: 0.0712 - val_loss: 0.2276


In [48]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia6.h5')

Epoch 1/10
4264/4263 [==============================] - 5655s - loss: 0.0714 - val_loss: 0.2184
Epoch 2/10
4264/4263 [==============================] - 5445s - loss: 0.0711 - val_loss: 0.2192
Epoch 3/10
4264/4263 [==============================] - 4295s - loss: 0.0710 - val_loss: 0.2154
Epoch 4/10
4264/4263 [==============================] - 4295s - loss: 0.0708 - val_loss: 0.2165
Epoch 5/10
4264/4263 [==============================] - 4293s - loss: 0.0709 - val_loss: 0.2189
Epoch 6/10
4264/4263 [==============================] - 4304s - loss: 0.0708 - val_loss: 0.2138
Epoch 7/10
4264/4263 [==============================] - 7401s - loss: 0.0707 - val_loss: 0.2173
Epoch 8/10
4264/4263 [==============================] - 8149s - loss: 0.0707 - val_loss: 0.2172
Epoch 9/10
4264/4263 [==============================] - 8158s - loss: 0.0705 - val_loss: 0.2158
Epoch 10/10
4264/4263 [==============================] - 8169s - loss: 0.0705 - val_loss: 0.2194


## 3 Nvidia All CONV model

In [49]:
model = nvidia_conv()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_4 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 160, 320, 3)       12        
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 160, 320, 24)      672       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 80, 160, 24)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 80, 160, 36)       7812      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 40, 80, 36)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 40, 80, 48)        15600     
__________

In [50]:
model.compile(optimizer="adam", loss="mse")

In [51]:
# serialize model to JSON
model_path = results_path+'nvidia_conv.json'
model_json = model.to_json()
with open(model_path, "w") as json_file:
    json_file.write(model_json)

print("Saved model to disk")

Saved model to disk


In [52]:
trn_gen = generate_batch(x_trn.shape[0], batch_size)
val_gen = generate_batch(x_val.shape[0], batch_size, train=False)

In [53]:
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia_conv1.h5')

Epoch 1/10
4264/4263 [==============================] - 4772s - loss: 0.0604 - val_loss: 0.3078
Epoch 2/10
4264/4263 [==============================] - 4734s - loss: 0.0611 - val_loss: 0.3096
Epoch 3/10
4264/4263 [==============================] - 4725s - loss: 0.0617 - val_loss: 0.3573
Epoch 4/10
4264/4263 [==============================] - 4723s - loss: 0.0627 - val_loss: 0.3523
Epoch 5/10
4264/4263 [==============================] - 4718s - loss: 0.0644 - val_loss: 0.3407
Epoch 6/10
4264/4263 [==============================] - 4731s - loss: 0.0635 - val_loss: 0.3497
Epoch 7/10
4264/4263 [==============================] - 4719s - loss: 0.0629 - val_loss: 0.3447
Epoch 8/10
4264/4263 [==============================] - 4892s - loss: 0.0628 - val_loss: 0.3542
Epoch 9/10
4264/4263 [==============================] - 7126s - loss: 0.0622 - val_loss: 0.3541
Epoch 10/10
4264/4263 [==============================] - 7093s - loss: 0.0621 - val_loss: 0.3466


In [54]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia_conv2.h5')

Epoch 1/10
4264/4263 [==============================] - 6892s - loss: 0.0627 - val_loss: 0.0604
Epoch 2/10
4264/4263 [==============================] - 6885s - loss: 0.0623 - val_loss: 0.1067
Epoch 3/10
4264/4263 [==============================] - 6909s - loss: 0.0627 - val_loss: 0.1158
Epoch 4/10
4264/4263 [==============================] - 6992s - loss: 0.0625 - val_loss: 0.0430
Epoch 5/10
4264/4263 [==============================] - 7058s - loss: 0.0625 - val_loss: 0.1184
Epoch 6/10
4264/4263 [==============================] - 7040s - loss: 0.0631 - val_loss: 0.1159
Epoch 7/10
4264/4263 [==============================] - 7060s - loss: 0.0621 - val_loss: 0.1190
Epoch 8/10
4264/4263 [==============================] - 7078s - loss: 0.0635 - val_loss: 0.1093
Epoch 9/10
4264/4263 [==============================] - 7072s - loss: 0.0632 - val_loss: 0.1007
Epoch 10/10
4264/4263 [==============================] - 7125s - loss: 0.0622 - val_loss: 0.1081


In [55]:
model.optimizer.lr = 1e-3
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia_conv3.h5')

Epoch 1/10
 132/4263 [..............................] - ETA: 7586s - loss: 0.0900

KeyboardInterrupt: 

In [ ]:
#filepath2 = results_path+'nvidia_aug.h5'
#checkpoint2 = ModelCheckpoint(filepath2, monitor='val_loss', verbose=1, save_best_only=False,\
#                             save_weights_only=True, mode='min', period=1)
#callbacks2=[checkpoint2]

In [ ]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia_conv4.h5')

Epoch 1/10
   7/4263 [..............................] - ETA: 7907s - loss: 0.0880

In [ ]:
model.optimizer.lr = 1e-4
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia_conv5.h5')

In [ ]:
model.optimizer.lr = 1e-5
model.fit_generator(generator=trn_gen, steps_per_epoch=len(x_trn) / batch_size, epochs=10, verbose=1, \
                    validation_data=val_gen, validation_steps=len(x_val) / batch_size)

model.save_weights(results_path+'nvidia_conv6.h5')